# PPS Workbench (v1.0)
Convert a one-row-per-pair spreadsheet into the v1 JSON dataset, validate it, and try simple queries.
- **Input:** CSV/XLSX with columns: `id, ref, chapter, text_KJV, text_WEB(optional), input_tag, output_tag, type, anchor1, anchor2, note`
- **Output:** `/data/proverbs.json` grouped by verse/range with `pairs[]`.
- **Rules (v1.0):** positive, unambiguous **causal** pairs only; KJV (default) + WEB display.


In [ ]:
# 0) Imports & paths
import os, json, re
from dataclasses import dataclass
from typing import List, Dict, Any, Optional
import pandas as pd
from IPython.display import display, HTML

BASE = "/mnt/data"
DATA_DIR = os.path.join(BASE, "pps_docs_stub", "data")
INDEX_DIR = os.path.join(DATA_DIR, "indexes")
CONFIG_PATH = os.path.join(DATA_DIR, "config.json")
PROVERBS_JSON_PATH = os.path.join(DATA_DIR, "proverbs.json")
INPUT_PATH = ""  # <-- set to your CSV/XLSX path (uploaded to this runtime), e.g., '/mnt/data/your_file.xlsx'

print("Using data dir:", DATA_DIR)

In [ ]:
# 1) Load input sheet
if not INPUT_PATH:
    print("Set INPUT_PATH to your CSV/XLSX file path and re-run this cell.")
else:
    if INPUT_PATH.lower().endswith(".csv"):
        raw = pd.read_csv(INPUT_PATH)
    else:
        raw = pd.read_excel(INPUT_PATH)
    display(raw.head())

In [ ]:
# 2) Normalize & basic checks
required_cols = ["id","ref","chapter","text_KJV","input_tag","output_tag","type"]
optional_cols = ["text_WEB","anchor1","anchor2","note"]

def normalize_df(df: pd.DataFrame) -> pd.DataFrame:
    # Ensure columns exist
    for c in required_cols + optional_cols:
        if c not in df.columns:
            df[c] = ""
    # Strip whitespace
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    # Coerce chapter to int where possible
    def to_int(x):
        try:
            return int(float(x))
        except Exception:
            return None
    df["chapter"] = df["chapter"].apply(to_int)
    return df[required_cols + optional_cols]

def validate_rows(df: pd.DataFrame):
    id_re = re.compile(r"^Prov\.\d{2}\.\d{2}(-\d{2})?$")
    ref_re = re.compile(r"^Proverbs\s+\d+:\d+(-\d+)?$")
    errors = []
    for i, row in df.iterrows():
        row_errs = []
        if not id_re.match(str(row["id"])):
            row_errs.append("bad id")
        if not ref_re.match(str(row["ref"])):
            row_errs.append("bad ref")
        chap = row["chapter"]
        if chap is None or chap < 1 or chap > 31:
            row_errs.append("bad chapter")
        if not row["text_KJV"]:
            row_errs.append("empty text_KJV")
        if str(row["type"]).lower() != "causal":
            row_errs.append("type must be 'causal' for v1")
        if not row["input_tag"]:
            row_errs.append("missing input_tag")
        if not row["output_tag"]:
            row_errs.append("missing output_tag")
        for a in ["anchor1","anchor2"]:
            if row[a] and len(str(row[a])) > 120:
                row_errs.append(f"{a} too long (>120)")
        if row_errs:
            errors.append({"row": int(i), "issues": row_errs})
    return errors

if 'raw' in globals():
    df = normalize_df(raw.copy())
    errs = validate_rows(df)
    print(f"Rows: {len(df)}  |  Errors: {len(errs)}")
    if errs:
        display(pd.DataFrame(errs))

In [ ]:
# 3) Derive canonical tag lists (or load separately if you prefer)
def derive_tags(df: pd.DataFrame):
    inputs = sorted(set(df["input_tag"].dropna().astype(str)))
    outputs = sorted(set(df["output_tag"].dropna().astype(str)))
    input_tags = [{"id": t, "label": t.replace("_"," ").title()} for t in inputs if t]
    output_tags = [{"id": t, "label": t.replace("_"," ").title()} for t in outputs if t]
    return input_tags, output_tags

if 'df' in globals():
    input_tags, output_tags = derive_tags(df)
    print(f"Derived {len(input_tags)} input tags, {len(output_tags)} output tags")
    # Save for the site
    with open(os.path.join(DATA_DIR, "input_tags.json"), "w", encoding="utf-8") as f:
        json.dump(input_tags, f, ensure_ascii=False, indent=2)
    with open(os.path.join(DATA_DIR, "output_tags.json"), "w", encoding="utf-8") as f:
        json.dump(output_tags, f, ensure_ascii=False, indent=2)

In [ ]:
# 4) Group into verse-level JSON
def group_rows(df: pd.DataFrame) -> Dict[str, Any]:
    grouped = {}
    for _, r in df.iterrows():
        rid = r["id"]
        if rid not in grouped:
            grouped[rid] = {
                "id": rid,
                "ref": r["ref"],
                "text_KJV": r["text_KJV"],
                "text_WEB": r.get("text_WEB","") or None,
                "pairs": [],
                "note": r.get("note","") or None
            }
        pair = {
            "input_tag": r["input_tag"],
            "output_tag": r["output_tag"],
            "type": "causal",
            "anchor_phrases": [p for p in [r.get("anchor1",""), r.get("anchor2","")] if p]
        }
        grouped[rid]["pairs"].append(pair)
    # Sort by id Prov.CC.VV
    def sort_key(e):
        # handle ranges: Prov.CC.VV or Prov.CC.VV-VV
        parts = e.split(".")
        cc, vv = parts[1], parts[2]
        vv0 = vv.split("-")[0]
        return (int(cc), int(vv0))
    items = [grouped[k] for k in sorted(grouped.keys(), key=sort_key)]
    return {
        "meta": {"book": "Proverbs", "version": "1.0"},
        "data": items
    }

if 'df' in globals() and not errs:
    dataset = group_rows(df)
    with open(PROVERBS_JSON_PATH, "w", encoding="utf-8") as f:
        json.dump(dataset, f, ensure_ascii=False, indent=2)
    print("Wrote:", PROVERBS_JSON_PATH)
    print("Records:", len(dataset["data"]))

In [ ]:
# 5) Build tiny indexes (optional)
def build_indexes(dataset: Dict[str, Any]):
    by_tag = {}
    by_chap = {}
    for rec in dataset["data"]:
        # chapter inferred from ref "Proverbs C:V(-VV)"
        try:
            chap = int(rec["ref"].split()[1].split(":")[0])
        except Exception:
            chap = None
        if chap:
            by_chap.setdefault(str(chap), []).append(rec["id"])
        tags = set()
        for p in rec["pairs"]:
            tags.add(p["input_tag"])
            tags.add(p["output_tag"])
        for t in tags:
            by_tag.setdefault(t, []).append(rec["id"])
    return by_tag, by_chap

if 'dataset' in globals():
    by_tag, by_chap = build_indexes(dataset)
    with open(os.path.join(INDEX_DIR, "by_tag.json"), "w", encoding="utf-8") as f:
        json.dump(by_tag, f, ensure_ascii=False, indent=2)
    with open(os.path.join(INDEX_DIR, "by_chapter.json"), "w", encoding="utf-8") as f:
        json.dump(by_chap, f, ensure_ascii=False, indent=2)
    print("Indexes saved.")

In [ ]:
# 6) Simple query (v1.0 rules)
def query(dataset: Dict[str,Any], inputs=None, outputs=None, chapters=None):
    inputs = [t for t in (inputs or []) if t]
    outputs = [t for t in (outputs or []) if t]
    chapset = set([int(c) for c in chapters]) if chapters else None

    results = []
    for rec in dataset["data"]:
        # Chapter filter
        chap = int(rec["ref"].split()[1].split(":")[0])
        if chapset and chap not in chapset:
            continue

        # Tag logic
        verse_inputs = {p["input_tag"] for p in rec["pairs"]}
        verse_outputs = {p["output_tag"] for p in rec["pairs"]}

        if inputs and not set(inputs).issubset(verse_inputs):
            continue
        if outputs and not set(outputs).issubset(verse_outputs):
            continue

        results.append(rec)
    return results

def highlight(text:str, anchors:list):
    htext = text
    for a in anchors or []:
        if a and a in htext:
            htext = htext.replace(a, f"<b><u><span>{a}</span></u></b>")
    return htext

def render(results, translation="KJV", limit=10):
    rows = []
    for rec in results[:limit]:
        text = rec["text_KJV"] if translation=="KJV" else (rec.get("text_WEB") or rec["text_KJV"])
        # Use the first pair's anchors for preview; UI would show all
        anchors = []
        for p in rec["pairs"]:
            anchors.extend(p.get("anchor_phrases",[])[:2])
        h = highlight(text, anchors[:2])
        pills = " ".join([f"<code>{p['input_tag']} → {p['output_tag']}</code>" for p in rec["pairs"]])
        rows.append(f"<div><strong>{rec['ref']}</strong><br/>{h}<br/>{pills}</div><hr/>")
    display(HTML("\n".join(rows) if rows else "<i>No results.</i>"))

if 'dataset' in globals():
    # Example: empty query shows all (first 10)
    results = query(dataset, inputs=[], outputs=[], chapters=None)
    render(results, translation="KJV", limit=5)

> **Note:** BibleGateway multi-passage link builder and selection/bulk actions are deferred to v1.2. 
You can still prototype them here later in a separate section without affecting v1.0 data/output.
